# How to make SQL queries with Claude


In this notebook, we'll explore how to use Claude to generate SQL queries based on natural language questions. We'll set up a test database, provide the schema to Claude, and demonstrate how it can understand and translate human language into SQL queries.

## Setup

First, let's install the necessary libraries and setup our Anthropic client with our API key.

In [1]:
# Install the necessary libraries
%pip install anthropic --quiet

  Using cached anthropic-0.45.2-py3-none-any.whl.metadata (23 kB)
Using cached anthropic-0.45.2-py3-none-any.whl (222 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import the required libraries
from anthropic import Anthropic
import sqlite3

# Set up the Anthropic API client
client = Anthropic()
# MODEL_NAME = "claude-3-opus-20240229"
MODEL_NAME = "claude-3-haiku-20240307"

In [10]:
from dotenv import load_dotenv
# Load .env file
load_dotenv()

True

In [11]:
import os

In [12]:
api_key = os.getenv("ANTHROPIC_API_KEY")
# print(apikey)

## Creating a Test Database

We'll create a test database using SQLite and populate it with sample data:

In [20]:
# Connect to the test database (or create it if it doesn't exist)
conn = sqlite3.connect("test_db.db")
cursor = conn.cursor()

In [21]:
# Create a sample table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        department TEXT,
        salary INTEGER
    )
""")

# Insert sample data
sample_data = [
    (1, "John Doe", "Sales", 50000),
    (2, "Jane Smith", "Engineering", 75000),
    (3, "Mike Johnson", "Sales", 60000),
    (4, "Emily Brown", "Engineering", 80000),
    (5, "David Lee", "Marketing", 55000)
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

IntegrityError: UNIQUE constraint failed: employees.id

In [22]:
cursor

## Generating SQL Queries with Claude

Now, let's define a function to send a natural language question to Claude and get the generated SQL query:

In [24]:
# Define a function to send a query to Claude and get the response
def ask_claude(query, schema):
    prompt = f"""Here is the schema for a database:

{schema}

Given this schema, can you output a SQL query to answer the following question? Only output the SQL query and nothing else.

Question: {query}
"""

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[
            {"role": 'user', "content":  prompt,},
            {"role": "assistant", "content": "SELECT"} #prefill as the LL
        ]
    )
    return response.content[0].text

We'll retrieve the database schema and format it as a string:

In [16]:
# Get the database schema
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


Now, let's provide an example natural language question and send it to Claude:

In [25]:
# Example natural language question
question = "What are the names and salaries of employees in the Engineering department?"
# Send the question to Claude and get the SQL query
sql_query = ask_claude(question, schema_str)
print(sql_query)

 name, salary
FROM EMPLOYEES
WHERE department = 'Engineering';


## Executing the Generated SQL Query

Finally, we'll execute the generated SQL query on our test database and print the results:

In [18]:
# Prepend 'SELECT ' to the query
sql_query = "SELECT " + sql_query

# Execute the SQL query and print the results
results = cursor.execute(sql_query).fetchall()

for row in results:
    print(row)

OperationalError: near "Here": syntax error

Don't forget to close the database connection when you're done:

In [19]:
# Close the database connection
conn.close()